In [1]:
import nltk
import os
import csv
import sklearn
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelBinarizer
import sklearn_crfsuite as crfsuite
from sklearn_crfsuite import metrics

In [2]:
train_sents = list(nltk.corpus.conll2002.iob_sents('ned.train'))
dev_sents = list(nltk.corpus.conll2002.iob_sents('ned.testa'))
test_sents = list(nltk.corpus.conll2002.iob_sents('ned.testb'))

In [3]:
with open('/Users/ellyzamaripapas/Code/NER_Project/data/annotated_data/cebuano-pos-1.txt') as file:
    lines = [line.rstrip() for line in file]

In [4]:
main_lst=[]
temp_lst = []
no=1
for line in lines:
    if line=='':
        main_lst.append(temp_lst)
        temp_lst=[]
        print(f'Article: {no}')
        no = no+1
    else:
        temp_lst.append(line)

Article: 1
Article: 2
Article: 3
Article: 4
Article: 5
Article: 6
Article: 7
Article: 8
Article: 9
Article: 10
Article: 11
Article: 12
Article: 13
Article: 14
Article: 15
Article: 16
Article: 17
Article: 18
Article: 19
Article: 20
Article: 21
Article: 22
Article: 23
Article: 24
Article: 25
Article: 26
Article: 27
Article: 28
Article: 29
Article: 30
Article: 31
Article: 32
Article: 33
Article: 34
Article: 35
Article: 36
Article: 37
Article: 38
Article: 39
Article: 40
Article: 41
Article: 42
Article: 43
Article: 44
Article: 45
Article: 46
Article: 47
Article: 48
Article: 49
Article: 50
Article: 51
Article: 52
Article: 53
Article: 54
Article: 55
Article: 56
Article: 57
Article: 58
Article: 59
Article: 60
Article: 61
Article: 62
Article: 63
Article: 64
Article: 65
Article: 66
Article: 67
Article: 68
Article: 69
Article: 70
Article: 71
Article: 72
Article: 73
Article: 74
Article: 75
Article: 76
Article: 77
Article: 78
Article: 79
Article: 80
Article: 81
Article: 82
Article: 83
Article: 84
A

In [5]:
#testing for single line
import re
#algo for converting to tuple
tuple(filter(None,[re.sub('[\s_]','',x) for x in main_lst[0][1].split('-')]))

('Gitanggong', 'X', 'O')

In [6]:
final_train=[]
for article in main_lst:
    temp_lst=[]
    for item in article:
        temp_lst.append(tuple(filter(None,[re.sub('[\s_]','',x) for x in item.split(' ')])))
    final_train.append(temp_lst)

In [7]:
#splitting
training_data = final_train[0:2170]
testing_data = final_train[2170:2790]
dev_data = final_train[2790:]

In [8]:
def read_clusters(cluster_file):
    word2cluster = {}
    with open(cluster_file) as i:
        for line in i:
            word, cluster = line.strip().split('\t')
            word2cluster[word] = cluster
    return word2cluster


def word2features(sent, i, word2cluster):
    word = sent[i][0]
    postag = sent[i][1]
    features = [
        'bias',
        'word.lower=' + word.lower(),
        'word[-3:]=' + word[-3:],
        'word[-2:]=' + word[-2:],
        'word.isupper=%s' % word.isupper(),
        'word.istitle=%s' % word.istitle(),
        'word.isdigit=%s' % word.isdigit(),
        'word.cluster=%s' % word2cluster[word.lower()] if word.lower() in word2cluster else "0",
        'postag=' + postag
    ]
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.extend([
            '-1:word.lower=' + word1.lower(),
            '-1:word.istitle=%s' % word1.istitle(),
            '-1:word.isupper=%s' % word1.isupper(),
            '-1:postag=' + postag1
        ])
    else:
        features.append('BOS')

    if i > 1: 
        word2 = sent[i-2][0]
        postag2 = sent[i-2][1]
        features.extend([
            '-2:word.lower=' + word2.lower(),
            '-2:word.istitle=%s' % word2.istitle(),
            '-2:word.isupper=%s' % word2.isupper(),
            '-2:postag=' + postag2
        ])        

        
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.extend([
            '+1:word.lower=' + word1.lower(),
            '+1:word.istitle=%s' % word1.istitle(),
            '+1:word.isupper=%s' % word1.isupper(),
            '+1:postag=' + postag1
        ])
    else:
        features.append('EOS')

    if i < len(sent)-2:
        word2 = sent[i+2][0]
        postag2 = sent[i+2][1]
        features.extend([
            '+2:word.lower=' + word2.lower(),
            '+2:word.istitle=%s' % word2.istitle(),
            '+2:word.isupper=%s' % word2.isupper(),
            '+2:postag=' + postag2
        ])

        
    return features


def sent2features(sent, word2cluster):
    return [word2features(sent, i, word2cluster) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

word2cluster = read_clusters("/Users/ellyzamaripapas/Code/NER_Project/data/cebtenten.tsv")

In [9]:
sent2features(training_data[0], word2cluster)[0]

['bias',
 'word.lower=-docstart-',
 'word[-3:]=RT-',
 'word[-2:]=T-',
 'word.isupper=True',
 'word.istitle=False',
 'word.isdigit=False',
 '0',
 'postag=-X-',
 'BOS',
 '+1:word.lower=gitanggong',
 '+1:word.istitle=True',
 '+1:word.isupper=False',
 '+1:postag=-X-',
 '+2:word.lower=karon',
 '+2:word.istitle=False',
 '+2:word.isupper=False',
 '+2:postag=-X-']

In [10]:
X_train = [sent2features(s, word2cluster) for s in training_data]
y_train = [sent2labels(s) for s in training_data]

X_dev = [sent2features(s, word2cluster) for s in dev_data]
y_dev = [sent2labels(s) for s in dev_data]

X_test = [sent2features(s, word2cluster) for s in testing_data]
y_test = [sent2labels(s) for s in testing_data]

In [11]:
crf = crfsuite.CRF(
    verbose='true',
    algorithm='lbfgs',
    max_iterations=100
)

try:
    crf.fit(X_train, y_train, X_dev=X_dev, y_dev=y_dev)
except AttributeError:
    pass
predictions = crf.predict(X_test)

loading training data to CRFsuite: 100%|██████████| 2170/2170 [00:00<00:00, 3724.14it/s]


loading dev data to CRFsuite: 100%|██████████| 310/310 [00:00<00:00, 3576.00it/s]



Holdout group: 2

Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 0
0....1....2....3....4....5....6....7....8....9....10
Number of features: 54738
Seconds required: 0.255

L-BFGS optimization
c1: 0.000000
c2: 1.000000
num_memories: 6
max_iterations: 100
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

Iter 1   time=0.07  loss=28027.11 active=54738 precision=0.100  recall=0.111  F1=0.105  Acc(item/seq)=0.898 0.281  feature_norm=1.00
Iter 2   time=0.07  loss=26625.99 active=54738 precision=0.100  recall=0.111  F1=0.105  Acc(item/seq)=0.898 0.281  feature_norm=1.05
Iter 3   time=0.04  loss=19901.26 active=54738 precision=0.119  recall=0.132  F1=0.125  Acc(item/seq)=0.900 0.281  feature_norm=2.29
Iter 4   time=0.04  loss=17138.04 active=54738 precision=0.169  recall=0.185  F1=0.172  Acc(item/seq)=0.905 0.281  feature_norm=3.23
Iter 5   time=0.04  loss=15757.25 active=54738 pr

In [12]:
import joblib
import os

OUTPUT_PATH = "crf-model"
OUTPUT_FILE = "crf-model-yes-bisaya1"

if not os.path.exists(OUTPUT_PATH):
    os.mkdir(OUTPUT_PATH)

joblib.dump(crf, os.path.join(OUTPUT_PATH, OUTPUT_FILE))

['crf-model/crf-model-yes-bisaya1']

In [13]:
crf = joblib.load(os.path.join(OUTPUT_PATH, OUTPUT_FILE))
y_pred = crf.predict(X_test)

example_sent = training_data[3]

print("Sentence:", ' '.join(sent2tokens(example_sent)))
print("Predicted:", ' '.join(crf.predict([sent2features(example_sent, word2cluster)])[0]))
print("Correct:  ", ' '.join(sent2labels(example_sent)))

Sentence: Matod niya , lima ka tuig na siya nga nagbalik-balik sa Sugbo aron pagtabang sa pamilya sa maong mga bata .
Predicted: O O O O O O O O O O O B-LOC O O O O O O O O O
Correct:   O O O O O O O O O O O B-LOC O O O O O O O O O


In [14]:
labels = list(crf.classes_)
labels.remove("O")
y_pred = crf.predict(X_test)
sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)
from sklearn import metrics

# Flatten the y_test and y_pred sequences
y_test_flat = [item for sublist in y_test for item in sublist]
y_pred_flat = [item for sublist in y_pred for item in sublist]

# Print the classification report
print(metrics.classification_report(y_test_flat, y_pred_flat))

#print(metrics.flat_classification_report(y_test, y_pred, labels=sorted_labels))

              precision    recall  f1-score   support

       B-LOC       0.84      0.72      0.78       285
       B-ORG       0.81      0.62      0.70       240
     B-OTHER       0.62      0.12      0.20        69
       B-PER       0.90      0.89      0.90       401
       I-LOC       0.77      0.67      0.71       212
       I-ORG       0.85      0.76      0.80       312
     I-OTHER       0.38      0.12      0.18        76
       I-PER       0.90      0.86      0.88       182
           O       0.98      1.00      0.99     14140

    accuracy                           0.96     15917
   macro avg       0.78      0.64      0.68     15917
weighted avg       0.96      0.96      0.96     15917



In [15]:
import sklearn_crfsuite
import numpy as np

def display_crf_weights(crf, top=30):
    # Get the feature weights from the CRF model
    feature_weights = crf.state_features_

    # Extract the values from the dictionary
    weights = np.array(list(feature_weights.values()))

    # Sort the feature weights by absolute value
    sorted_indices = np.argsort(np.abs(weights))[::-1]

    # Select the top feature weights
    top_indices = sorted_indices[:top]

    # Display the top feature weights and their corresponding features
    for idx in top_indices:
        feature = list(feature_weights.keys())[idx]
        weight = weights[idx]
        print(f"Feature: {feature}, Weight: {weight}")

# Assuming 'crf' is your CRF model object
display_crf_weights(crf, top=30)

Feature: ('-1:word.lower=si', 'B-PER'), Weight: 3.155399
Feature: ('word[-2:]=00', 'O'), Weight: 3.030586
Feature: ('-1:word.lower=ni', 'B-PER'), Weight: 2.96648
Feature: ('+1:word.lower=milyones', 'O'), Weight: 2.95292
Feature: ('word.istitle=False', 'O'), Weight: 2.570202
Feature: ('word[-3:]=000', 'O'), Weight: 2.289671
Feature: ('-1:word.lower=sa', 'B-LOC'), Weight: 2.149426
Feature: ('-1:word.lower=sa', 'B-ORG'), Weight: 2.104438
Feature: ('BOS', 'O'), Weight: 2.101569
Feature: ('word.isupper=False', 'O'), Weight: 2.084649
Feature: ('-2:word.lower=lungsod', 'B-LOC'), Weight: 1.939988
Feature: ('0', 'O'), Weight: -1.928892
Feature: ('-2:word.lower=ni', 'B-PER'), Weight: 1.809416
Feature: ('-1:word.lower=ang', 'B-ORG'), Weight: 1.798014
Feature: ('-1:word.lower=niadtong', 'O'), Weight: 1.612579
Feature: ('-1:word.lower=kang', 'B-PER'), Weight: 1.604732
Feature: ('word.lower=ombudsman', 'B-ORG'), Weight: 1.60159
Feature: ('word.lower=sa', 'O'), Weight: 1.569776
Feature: ('EOS', 'O'),

In [16]:
import scipy
import sklearn_crfsuite
from sklearn.metrics import make_scorer, classification_report
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

# Create a custom CRFWrapper class to add the keep_tempfiles attribute
class CRFWrapper(sklearn_crfsuite.CRF):
    def __init__(self, keep_tempfiles=True, c1=0.1, c2=0.1, **kwargs):
        super().__init__(c1=c1, c2=c2, **kwargs)
        self.keep_tempfiles = keep_tempfiles

# Instantiate the custom CRFWrapper class with keep_tempfiles=True and default c1, c2 values
crf = CRFWrapper(
    keep_tempfiles=True,
    algorithm='lbfgs',
    max_iterations=50,
    all_possible_transitions=True
)
params_space = {
    'c1': scipy.stats.expon(scale=0.5),
    'c2': scipy.stats.expon(scale=0.05),
}

# Define f1_scorer
f1_scorer = make_scorer(sklearn_crfsuite.metrics.flat_f1_score,
                        average='weighted', labels=crf.classes_)


# Define RandomizedSearchCV
rs = RandomizedSearchCV(crf, params_space,
                        cv=3,
                        verbose=1,
                        n_jobs=-1,
                        n_iter=50,
                        scoring=f1_scorer)
# Fit the RandomizedSearchCV
rs.fit(X_train, y_train)

# Get the best estimator from RandomizedSearchCV
best_crf = rs.best_estimator_

# Predictions using the best estimator
predictions = best_crf.predict(X_test)

# Print the classification report
print(classification_report(y_test_flat, y_pred_flat))

Fitting 3 folds for each of 50 candidates, totalling 150 fits


KeyboardInterrupt: 